# Entanglement Detection and Quantification

Preparing multi-qubit entangled states, measuring entanglement via
concurrence, negativity, and entanglement witnesses.

In [ ]:
import numpy as np
import sys
sys.path.append('..')

from src.state_preparation import bell_state, ghz_state, w_state, cluster_state, product_state
from src.entanglement import concurrence, negativity, entanglement_entropy
from src.witnesses import construct_bell_witness, construct_ghz_witness, evaluate_witness, sweep_noise_witness
from src.tomography_utils import get_density_matrix, purity
from src.plotting import plot_density_matrix, plot_entanglement_metrics, plot_witness_vs_noise

## 1. State Preparation and Density Matrices

In [ ]:
states = {
    'Bell |Φ+>': bell_state('phi_plus'),
    'Product': product_state(2),
}

for name, circuit in states.items():
    rho = get_density_matrix(circuit)
    print(f"{name}: purity = {purity(rho):.4f}")
    plot_density_matrix(rho, title=name, filename=f'{name.replace(" ", "_")}.png')

## 2. Entanglement Metrics

In [ ]:
# two-qubit states: concurrence + negativity
two_qubit_states = {
    'Bell |Φ+>': bell_state('phi_plus'),
    'Bell |Ψ->': bell_state('psi_minus'),
    'Product': product_state(2),
}

print(f"{'State':<15} {'Concurrence':<14} {'Negativity':<12} {'Purity':<10}")
print('-' * 51)
for name, circuit in two_qubit_states.items():
    rho = get_density_matrix(circuit)
    C = concurrence(rho)
    N = negativity(rho, n_qubits=2)
    P = purity(rho)
    print(f"{name:<15} {C:<14.4f} {N:<12.4f} {P:<10.4f}")

In [ ]:
# multi-qubit states: negativity + entanglement entropy
multi_states = {
    'GHZ-3': ghz_state(3),
    'W-3': w_state(3),
    'Cluster-4': cluster_state(4),
}

print(f"{'State':<15} {'Negativity':<14} {'Ent. Entropy':<14} {'Purity':<10}")
print('-' * 53)
for name, circuit in multi_states.items():
    rho = get_density_matrix(circuit)
    n_q = circuit.num_qubits
    N = negativity(rho, n_qubits=n_q)
    S = entanglement_entropy(rho, n_qubits=n_q, partition_qubits=[0])
    P = purity(rho)
    print(f"{name:<15} {N:<14.4f} {S:<14.4f} {P:<10.4f}")

## 3. Entanglement Witnesses

In [ ]:
# Bell witness
W_bell = construct_bell_witness()

for name, circuit in two_qubit_states.items():
    rho = get_density_matrix(circuit)
    val = evaluate_witness(W_bell, rho)
    entangled = 'YES' if val < 0 else 'NO'
    print(f"{name}: <W> = {val:.4f} -> Entangled: {entangled}")

In [ ]:
# witness under noise
noise_levels = np.linspace(0, 1, 50)
bell_circuit = bell_state('phi_plus')
result = sweep_noise_witness(bell_circuit, W_bell, noise_levels)

# find critical noise level
for i, val in enumerate(result['witness_values']):
    if val >= 0:
        print(f"Entanglement lost at noise p = {result['noise_levels'][i]:.3f}")
        break

plot_witness_vs_noise(result['noise_levels'], result['witness_values'],
                       state_name='Bell |Φ+>')